In [23]:
from skimage import io
from skimage.transform import resize
import skimage
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from ast import literal_eval
from skimage.util import crop
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
import math
from scipy import ndimage
import traceback

In [24]:
img = pd.read_csv('img.csv')

In [25]:
img

,Unnamed: 0,id,width,height,set,file_name
0,0,a4ea732cd3d5948a,840,1024,train,train/a4ea732cd3d5948a.jpg
1,1,4bf43a7b2a898044,1024,683,train,train/4bf43a7b2a898044.jpg
2,2,1b55b309b0f50d02,1024,683,train,train/1b55b309b0f50d02.jpg
3,3,00c359f294f7dcd9,1024,680,train,train/00c359f294f7dcd9.jpg
4,4,04b5a37f762b0f51,768,1024,train,train/04b5a37f762b0f51.jpg
...,...,...,...,...,...,...
21773,21773,d13b7ec72bd5eed5,1024,683,train,train/d13b7ec72bd5eed5.jpg
21774,21774,0022933cdceee189,1024,554,train,train/0022933cdceee189.jpg
21775,21775,6029c75e0325d164,1024,768,train,train/6029c75e0325d164.jpg
21776,21776,0ebbecdc46b78d42,1024,681,train,train/0ebbecdc46b78d42.jpg


In [26]:
annot = pd.read_csv('annot_final.csv')

In [27]:
annot

,Unnamed: 0,id,image_id,utf8_string,points
0,0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525..."
1,1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,..."
2,2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6..."
3,3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9..."
4,4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,..."
...,...,...,...,...,...
1052349,1052349,0ebbecdc46b78d42_15,0ebbecdc46b78d42,.,"[295.65, -0.14, 295.65, 27.33, 267.47, 27.03, ..."
1052350,1052350,a37e1fb026b80a6d_1,a37e1fb026b80a6d,RÖR,"[331.69, 466.97, 749.0, 462.84, 749.0, 659.79,..."
1052351,1052351,a37e1fb026b80a6d_2,a37e1fb026b80a6d,Moderna,"[876.75, 287.61, 912.34, 285.63, 913.73, 293.9..."
1052352,1052352,a37e1fb026b80a6d_3,a37e1fb026b80a6d,Museet,"[913.53, 284.84, 944.77, 282.86, 945.76, 292.3..."


In [28]:
annot.iloc[559]['points'], annot.iloc[559]['utf8_string'], annot.iloc[559]['image_id']

('[202.26, 60.4, 231.01, 58.12, 231.82, 67.38, 202.91, 69.33]',
 'menu',
 '4974a1000a27a984')

In [29]:
def cropRectangle(points, image):
    angle = -math.tan((points[3] - points[1])/(points[2] - points[0]))/2 - math.tan((points[7] - points[5])/(points[6] - points[4]))/2
    rotated = ndimage.rotate(image, angle * 180 / math.pi)
    point1x = (points[0] - image.shape[0]/2) * math.cos(angle) - (points[1] - image.shape[1]/2) * math.sin(angle) + rotated.shape[0]/2
    point1y = (points[1] - image.shape[1]/2) * math.cos(angle) + (points[0] - image.shape[0]/2) * math.sin(angle) + rotated.shape[1]/2
    point2x = (points[4] - image.shape[0]/2) * math.cos(angle) - (points[5] - image.shape[1]/2) * math.sin(angle) + rotated.shape[0]/2
    point2y = (points[5] - image.shape[1]/2) * math.cos(angle) + (points[4] - image.shape[0]/2) * math.sin(angle) + rotated.shape[1]/2
    #print(point1x, point1y, point2x, point2y)
    final = rotated[int(min(point1y, point2y)):int(max(point1y, point2y)), int(min(point1x, point2x)):int(max(point1x, point2x))]
    if final.shape[0] > final.shape[1]:
        final = ndimage.rotate(final, 90)
    final = cv2.cvtColor(final, cv2.COLOR_BGR2GRAY)#grayscaling
    return final

In [ ]:
dataset_path = 'train_images'
pattern = re.compile("^[^\w\s]+$")#strings that are just puncations
words_in_img = None
i = 560
while i < 1000: 
    print(i)#if reaches error just skip image and continue next
    try:
    #if i in [128, 166, 406]:
      #i+=1
      #continue
        image_id = img.iloc[i]['id']
        image_name = image_id + '.jpg'
        image_path = os.path.join(dataset_path, image_name)
    #image = io.imread(image_path)
        image = cv2.imread(image_path)
        gray_ed = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#grayscaling
        norm_ed = (gray_ed - np.min(gray_ed)) / (np.max(gray_ed) - np.min(gray_ed))#normalizing

        words_temp = annot[annot['image_id'] == image_id].copy()
        words_temp = words_temp[words_temp['utf8_string'].str.match(pattern) == False].copy()#check and drop strings that are just punctuations
        if words_temp.empty:#make sure after removing the df is not empty
            i += 1
            continue

        words_temp_points = list(words_temp['points'])
        words_temp_id = list(words_temp['id'])

        id_count = 0
        temp_img_list = []
        set_len = len(words_temp_points)
        while id_count < set_len:
    #for x in words_temp_points:
            crop_coord = np.array(literal_eval(words_temp_points[id_count]))#to crop out each individual word
            cropped = cropRectangle(crop_coord, image)
      # convert each image of shape (32, 128, 1) without streching/distorting too much by scaling and padding
            h,w = cropped.shape
            if w < 10 or h < 10:
                id_count += 1
                continue
            cropped = cv2.resize(cropped, (int(w * 32 / h), 32), interpolation= cv2.INTER_LINEAR)
            if w >= 128:
                cropped = cv2.resize(cropped, (128, 32))
            else:
                w_pad = 128 - int(w * 32 / h)
                cropped = cv2.copyMakeBorder(cropped, 0, 0, 0, w_pad, cv2.BORDER_CONSTANT, None, 0)

            temp_img_list.append(cropped)
            id_count += 1
    except Exception:
        i += 1
        traceback.print_exc()
        continue

    id_count = 0
    for y in temp_img_list:
    #plt.imsave('testing_images_saved/' + words_temp_id[id_count] + '.jpg', y, cmap='gray')
        plt.imsave('cropped/' + words_temp_id[id_count] + '.jpg', y, cmap='gray')
        id_count += 1

    if words_in_img is None:
        words_in_img = words_temp.copy()
    else:
        words_in_img = pd.concat([words_in_img, words_temp])

    words_in_img.to_csv('words_in_img.csv')
    i += 1